In [ ]:
from evaluate import load
import json
import string
import copy
import re
from nltk import sent_tokenize
from rouge_score import rouge_scorer, scoring

In [ ]:
with open("/shared/eng/pj20/firas_data/test_datasets/results/asqa_sonnet_retrieval_top_5_base.json", "r") as f:
    data = json.load(f)['data']

In [25]:
def compute_rouge(data):
    """Main function for rouge scoring.
    If two references are provided,
    the best score is chosen for each instance.
    Args:
        data: requires field `output` and `answer` (or `annotations` for ASQA)
        metrics: list of evaluation metrics
    Returns:
        dictionary representation of rouge scores
    """
    def _rouge_calculation(hypotheses,
                        references1,
                        references2=[],
                        metrics=['rougeLsum']):

        if references2 == []:
            references2 = references1

        scorer = rouge_scorer.RougeScorer(metrics, use_stemmer=True)
        aggregator = scoring.BootstrapAggregator()

        for i in range(len(hypotheses)):
            scores1 = scorer.score(references1[i], hypotheses[i])
            scores2 = scorer.score(references2[i], hypotheses[i])
            if scores1['rougeLsum'].fmeasure > scores2['rougeLsum'].fmeasure:
                aggregator.add_scores(scores1)
            else:
                aggregator.add_scores(scores2)

        scores = {m: [] for m in metrics}

        for m in metrics:
            fmeasure = aggregator.aggregate()[m].mid.fmeasure
            scores[m].append(fmeasure)

        for m in scores:
            scores[m] = 100 * sum(scores[m]) / len(scores[m])

        return scores

    hypotheses = {}
    references1 = {}
    references2 = {}

    for idx, item in enumerate(data):
        hypotheses[idx] = item["output"]
        if "annotations" in item and item['annotations'] is not None: # For ASQA
            references1[idx] = item["annotations"][0]["long_answer"]
            references2[idx] = item["annotations"][1]["long_answer"]
        else:
            references1[idx] = item["answer"]
            references2[idx] = item["answer"]

    h, r1, r2 = [], [], []

    for key in references1:
        h.append(hypotheses[key])
        r1.append(references1[key])

        if references2 is not None:
            r2.append(references2[key])

    h = ['\n'.join(sent_tokenize(text.lower())) for text in h]
    r1 = ['\n'.join(sent_tokenize(text.lower())) for text in r1]
    r2 = ['\n'.join(sent_tokenize(text.lower())) for text in r2]
    scores = _rouge_calculation(h, r1, r2)

    return scores['rougeLsum']


def remove_citations(sent):
    return re.sub(r"\[\d+", "", re.sub(r" \[\d+", "", sent)).replace(" |", "").replace("]", "")


mauve = load('mauve')
    
normalized_data = copy.deepcopy(data)
for i in range(len(normalized_data)):
    normalized_data[i]['output'] = remove_citations(normalized_data[i]['output'])

references = [' '.join((item['question'] + " " + item['answer'].strip()).split()[:100]).rstrip(string.punctuation) for item in normalized_data]
predictions = [' '.join((item['question'] + " " + item['output'].strip()).split()[:100]).rstrip(string.punctuation) for item in normalized_data]

import mauve
mauve_results = mauve.compute_mauve(
    p_text=references,
    q_text=predictions,
    device_id=4,
    max_text_length=512,
    verbose=True,
    batch_size=32,
    featurize_model_name="gpt2-large"
)
print("MAUVE: ", mauve_results.mauve * 100)

rouge_results = compute_rouge(normalized_data)
print("ROUGE: ", rouge_results)


In [44]:
references[0]

"Who has the highest goals in world football? The players with the highest all-time goals and highest men's and women's international football goals differ. The player with the highest all-time men's football goals is Josef Bican, who in 2020 was recognized by FIFA, the international governing body of football, as the record scorer with an estimated 805 goals. Christine Sinclair has the highest goals in women's international football with 187 and is the all-time leader for international goals scored for men or women. Cristiano Ronaldo and Ali Daei are currently tied for leading goalscorer in the history of men's international"